# LiqPay API Notebook

Load environment variables and code autoreload plugin.

In [ ]:
%load_ext autoreload
%autoreload 2

%load_ext dotenv
%dotenv

Imports

In [ ]:
import json
import random
import string

from uuid import uuid4
from pprint import pprint
from datetime import date, datetime, timedelta, UTC

from liqpy.client import Client
from liqpy.exceptions import LiqPayException
from liqpy.testing import TestCard, gen_card_cvv, gen_card_expire

Get URL for callbacks.

In [ ]:
%env SERVER_URL

In [ ]:
SERVER_URL = _

Initialize client with public and private keys

In [ ]:
client = Client()
client

Encode request and decode it as callback

In [ ]:
data, signature = client.encode(
    action="pay",
    amount=1,
    order_id=str(uuid4()),
    description="Test Encoding",
    currency="USD",
    server_url=SERVER_URL,
)

sep, end = "\n", "\n\n"
print("data:", data, sep=sep, end=end)
print("signature:", signature, sep=sep, end=end)

client.callback(data, signature)

Pay request with test card

In [ ]:
card_exp_month, card_exp_year = gen_card_expire(valid=True)

order_id = str(uuid4())
client.request(
    "pay",
    order_id=order_id,
    amount=1,
    currency="USD",
    description="Test Payment",
    card=TestCard.successful(),
    card_exp_month=card_exp_month,
    card_exp_year=card_exp_year,
    card_cvv=gen_card_cvv(),
    server_url=SERVER_URL,
)

Pay request with fail

In [ ]:
try:
    card_exp_month, card_exp_year = gen_card_expire(valid=True)

    order_id = str(uuid4())
    client.request(
        "pay",
        order_id=order_id,
        amount=1,
        currency="USD",
        description="Test Payment",
        card=TestCard.failure("9859"),
        card_exp_month=card_exp_month,
        card_exp_year=card_exp_year,
        card_cvv=gen_card_cvv(),
        server_url=SERVER_URL,
    )
except LiqPayException as e:
    print(e.response)
    pprint(e.details)
    raise e

Get a payment status

In [ ]:
client.status(order_id)

add info to payment

In [ ]:
client.data(order_id, "Test info")

Create checkout link

In [ ]:
client.checkout(
    action="pay",
    amount=1,
    order_id=str(uuid4()),
    description="Test Checkout",
    currency="USD",
    server_url=SERVER_URL
)

Get payments report

In [ ]:
date_to = datetime.now(UTC)
date_from = date_to - timedelta(days=30)

data = client.reports(date_from=date_from, date_to=date_to, format="json")
data = json.loads(data)